In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
#Blas GEMM launch failed , 避免動態分配GPU / CPU, 出現問題
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 50 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
len_train= x_train.shape[0]
len_test = x_test.shape[0]
print('x_train shape:', x_train.shape)
print(len_train, 'train samples')
print(len_test, 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# 分批讀入compile
def cifar_generator(image_array, batch_size=100):
    while True:
        for indexs in range(0, len(image_array), batch_size): # range(start, stop[, step])
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels

In [5]:
def test_generator(image_array, batch_size=20):
    while True:
        for indexs in range(0, len(image_array), batch_size): # range(start, stop[, step])
            images = x_test[indexs: indexs+batch_size]
            labels = y_test[indexs: indexs+batch_size]
            yield images, labels

In [6]:
# 建立模型
keras.backend.clear_session() # 把舊的 Graph 清掉
model = Sequential()
model.add(Conv2D(32, (5, 5), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)       

In [7]:
cifar_gen = cifar_generator(x_train)
test_gen  = test_generator(x_test)

model.fit_generator(
                    cifar_gen,
                    steps_per_epoch= 100,
                    epochs=epochs,
                    verbose=1, 
                    validation_data = test_gen,
                    validation_steps = 100,
                    initial_epoch=0
                    )
    

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
100/100 [==============================] - 3s 30ms/step - loss: 2.0779 - accuracy: 0.2140 - val_loss: 2.1394 - val_accuracy: 0.3120
Epoch 2/50
100/100 [==============================] - 1s 14ms/step - loss: 1.7642 - accuracy: 0.3451 - val_loss: 1.6886 - val_accuracy: 0.3750
Epoch 3/50
100/100 [==============================] - 1s 13ms/step - loss: 1.5670 - accuracy: 0.4161 - val_loss: 1.3824 - val_accuracy: 0.4660
Epoch 4/50
100/100 [==============================] - 1s 13ms/step - loss: 1.4812 - accuracy: 0.4559 - val_loss: 1.3457 - val_accuracy: 0.4900
Epoch 5/50
100/100 [==============================] - 1s 13ms/step - loss: 1.4050 - accuracy: 0.4918 - val_loss: 1.5090 - val_accuracy: 0.5425
Epoch 6/50
100/100 [==============================] - 1s 13ms/step - loss: 1.3092 - accuracy: 0.5258 - val_loss: 1.6148 - val_accuracy: 0.5785
Epoch 7/50
100/100 [==============================] - 1s 13ms/step - loss: 1.2892 - accuracy: 0.5395 - val_loss: 1.2289 - val_accuracy: 0.5660